# Analyzing Regression Metrics

## Introduction
Regression is a foundational __tool in machine learning for predicting continuous variables__. Accurately __evaluating regression models is crucial to ensure their performance and reliability__. This document explores __common regression metrics__, their calculations, and their practical applications.

# Data and Setup

## Libraries Used

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Dataset

We use the California Housing dataset from sklearn, which provides housing-related features and median house values.

In [2]:
# Get the data 
data = fetch_california_housing()

# Getting the features and the target as Dataframes 
features_df = pd.DataFrame(data["data"], columns = data.feature_names)
target_df = pd.DataFrame(data["target"], columns = data.target_names)

# Merging both the features and the target into the same dataframe
housing_df = pd.concat( 
    [features_df, target_df],
    axis = 1 # Horizontally
)

# The target variable is express in hundreds of thousands, but 
# I would like to have the original value in the target variable
housing_df["MedHouseVal"] = housing_df["MedHouseVal"] * 100_000

In [3]:
print(f"Rows: {housing_df.shape[0]}")
print(f"Columns: {housing_df.shape[1]}")

print("Target:", data.target_names[0])

print(f"Features ({len(data.feature_names)}):")
for feature in data.feature_names:
    print(" -", feature)

print("")

example_nrows = 4
print(f"Example:")
housing_df.head(example_nrows)

Rows: 20640
Columns: 9
Target: MedHouseVal
Features (8):
 - MedInc
 - HouseAge
 - AveRooms
 - AveBedrms
 - Population
 - AveOccup
 - Latitude
 - Longitude

Example:


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,452600.0
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,358500.0
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,352100.0
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,341300.0


## Train-Test Split

Data is split into 80% training and 20% testing for model evaluation.

In [4]:
train_df, test_df = train_test_split(
    housing_df,
    test_size=0.2, 
    random_state=42
)

print("Train data set: ",  train_df.shape[0])
print("Test data set: ", test_df.shape[0])

Train data set:  16512
Test data set:  4128


# Regression Models

Just to test how different models get a different score, we are going to use a Linear Regression Model and a Random Forest model.

## Linear Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, 
    test_size=0.2, 
    random_state = 42
)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_regression_model = LinearRegression()
linear_regression_model.fit(
    X_train, y_train
)

In [ ]:
r_squared = linear_regression_model.score(X_train, y_train)

In [ ]:
print("R^2 = " , r_squared)

In [ ]:
predicted = linear_regression_model.predict( train_df[data.feature_names])

In [ ]:
linear_regression_model_df = LinearRegression()
linear_regression_model_df.fit(train_df, y_train)


In [ ]:
y_train

In [ ]:
train_df.head(3)

In [ ]:
lr = LinearRegression()
lr.fit(train_df[data.feature_names], train_df[data.target_names])


In [ ]:
train_df["predicted_lr_train"] = lr.predict(train_df[data.feature_names])
train_df.head(3)

In [ ]:
train_df["difference_train"] = train_df["MedHouseVal"] - train_df["predicted_lr_train"]

In [ ]:
train_df["difference_squared_train"] = train_df["difference_train"] ** 2

In [ ]:
train_df.head(2)

# $ R^2 $

## How to calculate it

The $ R^2 $ formula is defined as follows: 

$ R^2 =  1 - \frac{ SS_{res} }{ SS_{tot} }  $

Where:
- $ SS_{res} = \sum_{i=1}^{n} (y_i - \hat{y_i})^2  $
- $ SS_{tot} = \sum_{i=1}^{n} (y_i - \bar{y_i})^2  $
    - $ {y_i} $: Real Value
    - $ \hat{y_i} $: Predicted value 
    - $ \bar{y_i} $: Mean of observed values

Meaning:
- $ SS_{res} = \sum_{i=1}^{n} (y_i - \hat{y_i})^2  $ (Residuals sum of squares)
- $ SS_{tot} = \sum_{i=1}^{n} (y_i - \bar{y_i})^2  $ (Residuals considering the mean)


## Example with a small table
$$
\begin{array}{|c|c|c|}
\hline
Observed (y) & Predicted (\hat{y}) & Error (y - \hat{y}) \\ 
\hline
5.0 & 4.8 & 0.2 \\ 
\hline
7.0 & 6.5 & 0.5 \\ 
\hline
4.0 & 4.2 & -0.2 \\ 
\hline
\end{array}
$$ 


$ SS_{res} = \sum_{i=1}^{n} (y_i - \hat{y_i})^2 $
$ = (5-4.8)^2 + (7-6.5)^2 + (4 - 4.2)^2 $
$ = 0.04 + 0.25 + 0.04 $
$ = 0.33 $

To calculate $ SS_{tot} $ we need to first calculate the mean ($ \bar{y} $). So let's calculate the mean: 
$$ \bar{y} = \frac{5 + 7 + 4}{3} = \frac{16}{3} = 5.33 $$

Now, we can calculate the $ SS_{tot} $ 

$ SS_{tot} = \sum_{i=1}^{n} (y_i - \bar{y_i})^2  $ 
$ = (5 - 5.33)^2 + (7 - 5.33)^2 + (4 - 5.33)^2  $ 
$ =  0.1089 + 2.7889 + 1.7689$ 
$ =  4.67 $ 

Now that we have both terms calculated ( $ SS_{res} $ and $ SS_{tot} $ ) we can calculate $ R^2 $

$ R^2 = 1 - \frac{SS_{res}}{SS_{tot}} $ 
$ = 1 - \frac{0.33}{4.67} $ 
$ = 1 - 0.07 $ 
$ = 0.93 $

So we can say that the model explains approximately 93% of the variance in the observed data.


## $ R^2 $ in the training set

Now we want to calculate the value of $ R^2 $ applied to our training set. This means that we have used our machine learning model to predict on our train set (this doesn't make much sense because the model has been trained with the training set) but we can do this later on the test set and compare both results.

In [ ]:
train_df.head(2)

In [ ]:
# We need the mean of the observed values
mean_observed_values = np.mean(train_df["MedHouseVal"])
print(f"Mean of the house price: {mean_observed_values }")

Now, let's calculate the term: $ SS_{tot} = \sum_{i=1}^{n} (y_i - \bar{y_i})^2  $ 


In [ ]:
train_df["SS_tot"] =  (train_df["MedHouseVal"] - mean_observed_values)**2

In [ ]:
ss_tot = train_df["SS_tot"].sum()
print(f"SS_tot term: {ss_tot}")

Now, let's calculate the - $ SS_{res} = \sum_{i=1}^{n} (y_i - \hat{y_i})^2  $ (Residuals sum of squares)


In [ ]:
train_df["SS_res"] = (train_df["MedHouseVal"] - train_df["predicted_lr_train"]) ** 2


In [ ]:
ss_res = train_df["SS_res"].sum()
print(f"SS_res term: {ss_res}")


$ R^2 =  1 - \frac{ SS_{res} }{ SS_{tot} }  $

In [ ]:
r_squared = 1 - (ss_res / ss_tot)
print(f"R squared: {r_squared}")

We should get the same result if we use the built in function in the linear regression model "score"


In [ ]:
linear_regression = LinearRegression()
linear_regression.fit(train_df[data.feature_names], train_df[data.target_names])
score_sklearn = linear_regression.score(train_df[data.feature_names], train_df[data.target_names])
print(f"Score (R^2) calculated with sklearn built in function: {score_sklearn}")


In [ ]:
print(f"""
R squared: 
     Self calculated: {r_squared}
  Sklearn calculated: {score_sklearn}
""")

As we can see its the same result

Now we can try to our model to predict with unseen data (test set), and check how much $R^2$ we get

In [ ]:
r_squared_with_test_set = linear_regression.score(test_df[data.feature_names], test_df[data.target_names])
print(f"R^2 obtained on the test set: {r_squared_with_test_set}")

## Is our model good?

This depends on the context of our data and the specific of the domain we're working in. 
However, there are some general guidelines:

### General Interpretation
- $R^2$ = 1 => __Perfect fit__. The model explains 100% of the variance in the dependent variable.
- $R^2$ = 0 => __The model explains none of the variance.__
- $R^2$ < 0 (Negative) => __The model performs worse than a simple horizontal line__ (mean of the data). This indicates a poorly fitted model.

### Typical Ranges in Practice
1. __High__ $R^2$ (__0.7 to 1.0__):
    - Indicates the __model explains a large portion of the variance.__
    - Often seen in __controlled experiments or well-understood domains, like physics__.
  
2. __Moderate__ $R^2$ (__0.4 to 0.7__):
    - Common in __fields like social sciences, biology, or economics.__
    - Is __acceptable if the data is inherently noisy or complex__.

3. __Low__ $R^2$ (__< 0.4)__:
    - Indicates the model has __limited explanatory power.__
    - Can be __acceptable in domains where many unmeasured factors influence the outcome (e.g. human behaviour studies).__

### Training vs Test
- __Training__ $R^2$: Measures fit on the data the model was trained on. __Higher scores are expected__.
- __Test__ $R^2$: Measures __generalization to unseen data__. If this __value is significantly lower that in the training, it might indicate overfitting.__

### Final Thoughs
1. __Compare $R^2$ against Benchmarks__: __Use domain-specific benchmarks or baseline models__. For instance, __compare $R^2$ with a simple linear regression__ model __or a previous model__ in the same task.
    
2. Consider Adjusted $R^2$:
    - Ensure the model ins't artificially inflating $R^2$ by adding irrelevant predictors.
    
3. Domain Expectations:
    - A "good" $R^2$ in physics might be 0.9 but in social sciences, 0.3 might be impressive due to data variability.


## $R^2$ Summary
- __Easy to interpret__ as __percentage of explained variance__.
- __Can be artificially inflated with useless variables__: new variables can either do nothing or explain more.
- Does not really tell us if the model is good or bad, models can have a high $R^2$ but still have large residuals (errors) => Explain most of the variance, but predictions still far off in absolute error.

# Adjusted $R^2$

$ R^2_{adj} = 1 - (1 - R^2) \frac{n - 1}{n - p - 1}  $

Where:
- p: Number of variables, so the more variables you add the bigger is the denominator (penalizes a lot of variable).
- n: Sample size. 

With this formula, we are "fixing" the problem that the $R^2$ has (artificially increase if we add more variable, even if the variables are useless). We do this by penalizing the number of variables that we have into account. 

As we can see in the formula, before calculating the $R^2_{adj}$ we need to calculate $R^2$ as it is one of the terms of the formula.

$R^2_{adj}$ can be negative, and it is always less than or equal to $R^2$. Unlike $R^2$, the $R^2_{adj}$ increases only when the increase of $R^2$ is more than one would expect by chance.

$R^2_{adj}$ Summary

- Penalizes unnecessary complexity (number of predictors)
- Better for comparing models with different number of variables (will only improve with usefull variables).
- Use $R^2$ and $R^2_{adj}$ in combination with other metrics.

Sklearn doesn't have a function to calculate $R^2_{adj}$ so we have to do the formula ourselves.

In [ ]:
# Let's get the R^2 that we got in the model
r_squared

In [ ]:
number_of_rows, columns = train_df.shape[0], train_df.shape[1]
n = number_of_rows
# The train dataset includes the target variable, so to get only the features we need
# to substract 1
p = columns - 1 

print(f"rows (n): {n}")
print(f"features (p): {columns}")

In [ ]:
r_squared_adjusted = 1 - (1 - r_squared) * ( (n - 1) / (n - p - 1) )
print(f"""
r^2: {r_squared}
r^2_adjusted: {r_squared_adjusted}
difference (r^2 - r^2_adjusted): {r_squared - r_squared_adjusted}
""")

# Regression Metrics

## MSE (Mean Squared Error)

$ \frac{1}{n} \sum^{a}_{i=1} (y_{i} - \hat{y}_{i})^2$

where: 
- $y_{i} - \hat{y}_{i}$: is the difference between the real value and the prediction done. We squared this term to avoid negative numbers.

What we are doing here is calculating the difference between the predicted value and the actual value, and summing up all this differences. 
- __Not very intuitive__ because as we have squared the differences, the output is not in the original scale (RMSE has the scale of the original measure).
- __Emphasizes large errors__ due to the squaring, useful when larger errors are unacceptable.
- __Useful in optimization algorithms__.
- __Good when trying to minimize overall error__ in the model.
- Highly __sensitive to outliers__.
- __Scale-Dependent__ (depends on the scale of the features)

In [ ]:
# Imaging that we have the following data:
true_values = [3, -0.5, 2, 7]
predicted_values = [2.5, 0.0, 2, 8]

# Let's calculate the MSE manually 
difference_between_real_and_predicted = np.array(true_values) - np.array(predicted_values)
squared_differences = difference_between_real_and_predicted ** 2
sum_squared_diffences = np.sum(squared_differences)
mse = 1/len(true_values) * sum_squared_diffences 

print(f"Difference between real and predicted: {difference_between_real_and_predicted}")
print(f"Squared differences: {squared_differences}")
print(f"Sum Squared Differences: {sum_squared_diffences}")
print(f"MSE:{mse}")

# We can also use sklearn to calculate the MSE
from sklearn.metrics import mean_squared_error
mse_sklearn = mean_squared_error(
    np.array(true_values),
    np.array(predicted_values)
)
print(f"MSE (with sklearn): {mse_sklearn}")


In [ ]:
true_values

In [ ]:
true_values = np.array(true_values)
predicted_values = np.array(predicted_values)
residuals = true_values - predicted_values
print(residuals)

In [ ]:
predicted_values

In [ ]:
# We can represent graphically the differences between the real
# and predicted values
def show_true_and_predicted(
    true_values, 
    predicted_values
):
    """
    Show in a graph the true vs predicted
    """
    
    # Create an array as long as the number of data points
    x = np.arange(len(true_values))
    
    plt.scatter(
        x, true_values, color="blue", label="True Values", marker= "v"
    )
    
    plt.scatter(
        x, predicted_values, color="red", label="Predicted Values"
    )
    
    plt.legend()

show_true_and_predicted(true_values, predicted_values)

In [ ]:
first_rows = train_df[["MedHouseVal", "predicted_lr_train"]].head(5)

In [ ]:
first_rows

In [ ]:
# If we use the training set to calculate the MSE
mse_training = 1/train_df.shape[0] *  np.sum(train_df["difference_squared_train"])
print(f"MSE training: {mse_training}")

In [ ]:
rmse_training = mse_training ** 1/2
print(f"RMSE training:{rmse_training}")

## RMSE (Root Mean Squared Error)

$ \sqrt{MSE} = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 }  $

- __Same units as response variable__, therefore __easy to interpret__.
- __Penalizes larger error due to squaring__.
- __Good when trying to minimize overall error in the model__.
- __Sensitive to outliers__.

## MAE (Mean Absolute Error)

$ \frac{1}{n} \sum_{i=1}^{n} {|(y_i - \hat{y_i})|} $

- Measures how wrong predictions were on average in original units, therefore its easy to understand for humans => "On average our model misses the correcty value by X"
- Less sensitive to outliers than MSE and RMSE
- Does not penalize large errors as heavily.
- Not differentiable at zero, which can affect some optimization methods => we cannot use MAE for some optimization algorithms.

In [ ]:
# Imaging that we have the following data:
true_values = [3, -0.5, 2, 7]
predicted_values = [2.5, 0.0, 2, 8]

manually_calculated_mae = 1/len(true_values)* sum(abs(np.array(true_values) - np.array(predicted_values)))


from sklearn.metrics import mean_absolute_error

sklearn_calculated_mae =  mean_absolute_error(true_values, predicted_values)

print(f"Manually calculated MAE: {manually_calculated_mae}")
print(f"Sklearn calculated MAE: {sklearn_calculated_mae}")


In [ ]:
def manually_MAE(true_values, predicted_values):
    return 1/len(true_values)* sum(abs(np.array(true_values) - np.array(predicted_values)))

In [ ]:
# If we calculated the MAE to our training set
print(manually_MAE(train_df["MedHouseVal"].values, train_df["predicted_lr_train"].values))
print(mean_absolute_error(train_df["MedHouseVal"].values, train_df["predicted_lr_train"].values))

print(f"""On average our model the correct value by {manually_MAE(train_df["MedHouseVal"].values, train_df["predicted_lr_train"].values)}""")

## MAPE (Mean Absolute Percentage Error)

$ \frac{1}{n} \sum_{i=1}^{n} |\frac{(y_i - \hat{y}_i)}{y_i}| $

- Expresses errors as percentages, making iterpretability better.
- Usefull when comparing forecast accuracy between datasets.
- Undefined when actual values are zero or near zero (denominator 0 is the fraction).

Here we don't have the actual units, but we have on average how much error in terms of percentage. 

This makes it more interpretable oftentimes, but also more useful when it comes to comparing it with other datasets or across datasets, because then it doesn't really matter 
what units ir or what the scale is, is about the percentage.

In [ ]:
# Imaging that we have the following data:
true_values = [3, -0.5, 2, 7]
predicted_values = [2.5, 0.0, 2, 8]

manually_calculated_mape = 1/len(true_values)* sum(abs((np.array(true_values) - np.array(predicted_values)) / np.array(true_values)))

from sklearn.metrics import mean_absolute_percentage_error

sklearn_calculated_mape =  mean_absolute_percentage_error(true_values, predicted_values)

print(f"Manually calculated MAPE: {manually_calculated_mape}")
print(f"Sklearn calculated MAPE: {sklearn_calculated_mape}")

## MedAE (Median Absolute Error)
$ MedAE = median(|y_i - y_i^2|) $

- Robust to outliers.
- Reflects typical magnitude of errors.
- May not capture effect of large errors.
- Less sensitive to overall error distribution

This is the same idea, but instead of taking the average, we take median values, of course this is much more robust when it comes to outliers, but in general it doesn't really capture any effect of large errors or outliers, it is really not sensitive to the error distribution because it doesn't really matter what's happening left and right, the important thing is what's in the center. In the center is located my typical magnitude of error and that's what I'm interest in, but wer're not understanding anything about the error distribution, maybe there is some huge error that I'm not seeing because I'm just looking at the median absolute error so that's of course a blind spot, but its more robuts to outliers.

In [ ]:
from sklearn.metrics import median_absolute_error

In [ ]:
# Imaging that we have the following data:
true_values = [3, -0.5, 2, 7]
predicted_values = [2.5, 0.0, 2, 8]

manually_calculated_medae = np.median(abs(np.array(true_values) - np.array(predicted_values)))

sklearn_calculated_medae =  median_absolute_error(true_values, predicted_values)

print(f"Manually calculated MedAE: {manually_calculated_medae }")
print(f"Sklearn calculated MedAE: {sklearn_calculated_medae }")

## Honorable Mentions
- MSLE (Mean Squared Logaritmic Error).
- RMSLE (Root Mean Squared Logaritmic Error).
- Explained Variance Score.
- Symmetric MAPE (Mean Absolute Percentage Error).
- Huber Loss.
- AIC and BIC (Akaike and Bayes Information Criteria).